# Finance Demo: Capital Allocation as an Evolutionary Game

This notebook demonstrates the finance pipeline:
1. Load/construct a quarterly **uses-of-cash** dataset `X` (items × time),
2. Remove **accounting identities** with `project_out_identities`,
3. Build a nonnegative strategy basis `S` (via simple NMF),
4. Construct a **value-gradient payoff** series `v` using a target `J_future`,
5. Fit `A` with `estimate_A_from_series`, and
6. Search for **ESS** with `find_ESS`.


In [ ]:
# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append('/mnt/data')
from ts2eg import (
    project_out_identities, value_gradient_payoffs,
    estimate_A_from_series, find_ESS
)
print('Ready.')

## 0) Synthetic quarterly financials (uses of cash)
Columns represent items: Revenue share allocated to {COGS, SG&A, R&D, CapEx, NWC change}.
We generate a plausible time series with slow drift + noise. Replace this with real 10-Q/10-K data.

In [ ]:
rng = np.random.default_rng(1)
T = 60   # quarters
items = ['COGS','SGA','R&D','CapEx','NWC']
N = len(items)
base = np.array([0.55, 0.20, 0.07, 0.05, 0.02])
trend = np.vstack([
    0.01*np.sin(2*np.pi*np.arange(T)/20 + 0.0),
    0.01*np.cos(2*np.pi*np.arange(T)/24 + 1.0),
    0.006*np.sin(2*np.pi*np.arange(T)/28 + 2.0),
    0.005*np.cos(2*np.pi*np.arange(T)/32 + 0.5),
    0.004*np.sin(2*np.pi*np.arange(T)/16 + 0.7),
])
noise = 0.01 * rng.standard_normal((N, T))
X = base.reshape(N,1) + trend + noise
X = np.clip(X, 0, None)
X = X / (X.sum(axis=0, keepdims=True) + 1e-12)   # shares of Revenue
print('X shape:', X.shape)

## 1) Remove identities (example)
Suppose we accidentally included 'GrossMargin' = 1 - COGS. Demonstrate projection that removes such redundancy.
Here we **don’t** add that row, but show the API: build C with rows c^T x = 0 and call `project_out_identities(X, C)`.

In [ ]:
# Example: if GrossMargin were included, a row c would encode: GM + COGS - 1 = 0 (after common-sizing)
C = np.zeros((0, N))   # no identities in this synthetic example
if C.shape[0] > 0:
    X = project_out_identities(X, C)
X_mean = X.mean(axis=1)
pd.DataFrame(X, index=items).head()

## 2) Build a nonnegative strategy basis S via simple NMF
We implement a tiny multiplicative-update NMF locally (no external deps).

In [ ]:
def nmf_multiplicative(V, r=3, iters=400, seed=0):
    rng = np.random.default_rng(seed)
    N, T = V.shape
    W = np.maximum(rng.random((N, r)), 1e-6)
    H = np.maximum(rng.random((r, T)), 1e-6)
    for _ in range(iters):
        WH = W @ H + 1e-12
        H *= (W.T @ (V / WH)) / (W.T @ np.ones_like(V) + 1e-12)
        WH = W @ H + 1e-12
        W *= ((V / WH) @ H.T) / (np.ones_like(V) @ H.T + 1e-12)
        W = np.maximum(W, 1e-12); H = np.maximum(H, 1e-12)
    return W, H

# Nonnegative scaling of X (already >=0 and columns sum to 1). Still normalize columns of S.
K = 3
S, H = nmf_multiplicative(X, r=K, iters=300, seed=2)
S = S / (np.linalg.norm(S, axis=0, keepdims=True) + 1e-12)
print('S shape:', S.shape)

## 3) Build a value target J and compute value-gradient payoffs v
We synthesize a target `J_future` as a 4-quarter-ahead performance proxy driven by a linear combo of items plus noise.

In [ ]:
true_beta = np.array([-0.6, -0.2, 0.5, 0.3, -0.1])
J = true_beta @ X + 0.05 * np.random.default_rng(3).standard_normal(T)
# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

from ts2eg import value_gradient_payoffs
v = value_gradient_payoffs(X, J, model='ridge', ridge=1e-2, standardize=True)
print('v shape:', v.shape)

## 4) Estimate A and search for ESS

In [ ]:
# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

from ts2eg import estimate_A_from_series, find_ESS
est = estimate_A_from_series(S, X, v, k=K, lambda_=1e-2)
A = est['A']
print('R^2:', round(est['R2'], 3))
ess = [r for r in find_ESS(A, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count:', len(ess))
for r in ess:
    print('ESS support:', r['support'], 'x*=', np.round(r['x'], 3))

## 5) Plots

In [ ]:
plt.figure(figsize=(7,3))
for i in range(K):
    plt.plot(est['Xk'][i], label=f'x_{i+1}')
plt.title('Inferred strategy memberships x(t)'); plt.legend(); plt.show()

import pandas as pd
print('Strategy basis S (items x K):')
display(pd.DataFrame(S, index=items, columns=[f's{i+1}' for i in range(K)]).round(3))